This is the second code file in the vaxxmisinfo project

<b> Important notes before you start: </b> 
<br><br>
Please make sure to modify the <b>  file paths </b> below to your project folder.
<br>
Please make sure to modify the <b>  value of the undersampling  </b> to ensure that the correct sampling value is used. 
<br>
<b> Pertaining to model training in section 4:</b> 
<br><br>
Please note that this may take a long time (depending on how many trianing samples you have). The function <b> best_model_search() </b> which trains the models currently uses n_jobs = -1 which utilizes all the cores / threads of your computer's CPU.Should you wish to not allocate the whole CPU towards model training (which is recommended), please adjust the n_jobs to be equal to the number of threads that you can sufficiently allocate. for example, if you have a CPU with 4 threads, you may want to allocate only half of these to the training process, hence you would set n_jobs = 2. 
<br><br>
To check how many threads you have please check "Task Manager" on Windows OS devices, or "About this Mac" on Mac OS devices

# 1. Importing Modules

In [ ]:
#Basic packages
import pandas as pd
import numpy as np
import os.path
import time
import pickle
import matplotlib.pyplot as plt
import seaborn as sb
import pickle
import matplotlib
from numpy import int64
from collections import Counter
import seaborn as sns

##### File paths

In [ ]:
project_folder = r'C:\Users\Farhan\Google Drive\Vaxxmissinfoproject'

#This is the path where all your annotated data files are
data_path = project_folder + '\Qualitative annotation\combined\Master' #<-Modify

#The path where the .sav files will be dumped as the models get trained
model_path = project_folder + '\Model Dumps testing' #<-Modify
 
#The path where the the final files with predictions for the dashboard are outputted. 
dashboard_file_path = project_folder + '\Dashboard\pred_data'  #<-Modify

#This is the path to the annotated data
annotated_data_path = project_folder +'\Output_Data_Files\converted_to_url\master_twitter_database.xlsx'  #<-Modify

#This is the path to the best selected model for final model deployment. 
#This will need to be updated based on your model's results. 
#Default is M3's Logistic Regression using TFIDF (based on best F1 Score)
best_model = model_path + '\m3\model_logistic_regression_tfidf_f1_macro.sav'  #<-Modify

##### Sampling values

In [ ]:
#Sampling values. Defaulted to 200

m1_sampling = 200  #<-Modify
m2_sampling = 200  #<-Modify
m3_sampling = 200  #<-Modify
m4_sampling = 200  #<-Modify
m5_sampling = 200  #<-Modify

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Used for webscraping
import re
import requests
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [ ]:
#SKLearn packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split # Splitting Data Sets
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [ ]:
#Classifiers
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#NLTK 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import *
nltk.download("stopwords")
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator  

In [ ]:
global_start_time = time.time() #This is so that you can see how long the program took (for troubleshooting)

# 2. Defining Global Functions

##### modify n_jobs if needed 

In [ ]:
# Modify n_jobs if needed

def best_model_search(X, y, vec1, vec2, model_params,model_dump_path):
    scores = list()
    estimators = list()
    vect = None

    for vec in [vec1,vec2]:
        X = X 
        y = y
        X_ft = vec.fit_transform(X).toarray()

        if vec == bow:
            vect = "bag-of-words"
        else:
            vect = "tfidf"

        for measure in ['accuracy','f1_macro','precision_macro','recall_macro']:
            for model_name, mp in model_params.items():
                print ('Current Model:', model_name, ' with ', vect, ' measure: ', measure)
                clf =  GridSearchCV(mp['model'], mp['params'], cv=7, return_train_score=True, scoring = measure, verbose=10, n_jobs =-1)
                clf.fit(X_ft, y)
                scores.append({
                    'model': model_name,
                    'vect': vect,
                    'score_method': measure,
                    'best_score': clf.best_score_,
                    'best_params': clf.best_params_
                })
                print ('Finished Model:', model_name, ' with ', vect, ' measure: ', measure,'best score: ', clf.best_score_)
                
                
                
                filename = model_dump_path + '\\model_' + model_name + '_' + vect + '_' + measure +'.sav'
                #pickle.dump(clf, open(filename, 'wb'))
                pickle.dump(clf.best_estimator_, open(filename, 'wb'))
                if measure == 'accuracy':
                    estimators.append(clf.best_estimator_)

            df = pd.DataFrame(scores,columns=['model','vect','score_method','best_score','best_params'])
    return df, estimators

In [ ]:
def clean_col_text(url_text,df):
    
    twitter_list = []
    stop_words = nltk.corpus.stopwords.words('english')

    for row in url_text:
        twitter_text = re.sub('(@[\w]+)|(^rt\s+)|(http[s]:\/\/[\w\.-\/]+[\s])|(http:\/\/[\w\.-\/]+[\s])|(#)|(â)','',row) #Remove html attributes 
        twitter_text = re.sub(r"http\S+", "", twitter_text) #Remove URL
        twitter_text = re.sub('([^\w]+)',' ',twitter_text) #Remove punctuation
        twitter_text = re.sub('(\s)x\w+|xe2',' ',twitter_text) #Remove emojis
        twitter_text = twitter_text.lower() #Lowercase all characters
        twitter_tokenizer = RegexpTokenizer(r'\w+') #Setup tokenizer          
        twitter_token = twitter_tokenizer.tokenize(twitter_text) #Tokenize words in string
        twitter_text = [word for word in twitter_token if word not in stop_words] #Remove stop words

        ps = PorterStemmer()
        twitter_text = [ps.stem(word) for word in twitter_text] #Stem 
        twitter_list.append(' '.join(twitter_text).replace('titl',''))

    df.insert(loc = len(df.columns), column = url_text.name + '_cleaned', value = twitter_list)

    return df

In [ ]:
def undersample_majority_classes(df, us_count):

    if df.target_label.nunique() == 5:
        
        df_class_0 = df[df['target_label'] == 0]
        df_class_1 = df[df['target_label'] == 1]
        df_class_2 = df[df['target_label'] == 2]
        df_class_3 = df[df['target_label'] == 3]
        df_class_4 = df[df['target_label'] == 4]
        count_class_0 = len(df_class_0.index)
        count_class_1 = len(df_class_1.index)
        count_class_2 = len(df_class_2.index)
        count_class_3 = len(df_class_3.index)
        count_class_4 = len(df_class_4.index)

        #print('Number of 0 labels', count_class_0)
        #print('Number of 1 labels', count_class_1)
        #print('Number of 2 labels', count_class_2)
        #print('Number of 3 labels', count_class_3)
        #print('Number of 4 labels', count_class_4)
        #df.target_label.value_counts().plot(kind='bar', title='Count (target)');

        df_class_0_under = df_class_0.sample(us_count)

        df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

        print('Random under-sampling:')
        #df_test_under.target_label.value_counts().plot(kind='bar', title='Count (target)');
        df1 = pd.concat([df_class_0_under, df_class_1], axis=0)
        df1 = pd.concat([df1, df_class_2], axis=0) 
        df1 = pd.concat([df1, df_class_3], axis=0) 
        df1 = pd.concat([df1, df_class_4], axis=0) 
        print('Total rows in df: ',len(df1.index))
        print (df1.target_label.value_counts().sort_index(axis=0))
        df1.target_label.value_counts().plot(kind='bar', title='Count (target)');

    elif df.target_label.nunique() == 4:
        df_class_0 = df[df['target_label'] == 0]
        df_class_1 = df[df['target_label'] == 1]
        df_class_2 = df[df['target_label'] == 2]
        df_class_3 = df[df['target_label'] == 3]
        
        count_class_0 = len(df_class_0.index)
        count_class_1 = len(df_class_1.index)
        count_class_2 = len(df_class_2.index)
        count_class_3 = len(df_class_3.index)
        
        df_class_0_under = df_class_0.sample(us_count)

        df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

        print('Random under-sampling:')
       
        df1 = pd.concat([df_class_0_under, df_class_1], axis=0)
        df1 = pd.concat([df1, df_class_2], axis=0) 
        df1 = pd.concat([df1, df_class_3], axis=0) 
        print('Total rows in df: ',len(df1.index))
        print (df1.target_label.value_counts().sort_index(axis=0))
        df1.target_label.value_counts().plot(kind='bar', title='Count (target)');
    
    elif df.target_label.nunique() == 3:
        df_class_0 = df[df['target_label'] == 0]
        df_class_1 = df[df['target_label'] == 1]
        df_class_2 = df[df['target_label'] == 2]
        
        count_class_0 = len(df_class_0.index)
        count_class_1 = len(df_class_1.index)
        count_class_2 = len(df_class_2.index)
        
        df_class_0_under = df_class_0.sample(us_count)

        df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

        print('Random under-sampling:')
        
        df1 = pd.concat([df_class_0_under, df_class_1], axis=0)
        df1 = pd.concat([df1, df_class_2], axis=0) 
        print('Total rows in df: ',len(df1.index))
        print (df1.target_label.value_counts().sort_index(axis=0))
        df1.target_label.value_counts().plot(kind='bar', title='Count (target)');
        
    return df1

In [ ]:
def plot_histogram_data(dataframe):
    a4_dims = (10, 5)
    fig, ax = plt.subplots(figsize=a4_dims)
    plt.xticks(rotation=45)
    sb.barplot(x='model', y='best_score',
    data=dataframe,hue = 'score_method').set_title(f"Model Comparison by {dataframe.iloc[0,1]}")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout()

If you would like to remove any stopwords from the feature set, add then here seperated by a comma

In [ ]:
additional_stopwords = ['amp']

In [ ]:
def clean_X(X):
    
    twitter_list = []
    stop_words = nltk.corpus.stopwords.words('english') + additional_stopwords

    for row in X:
        twitter_text = re.sub('(@[\w]+)|(^rt\s+)|(http[s]:\/\/[\w\.-\/]+[\s])|(http:\/\/[\w\.-\/]+[\s])|(#)|(â)','',row) #Remove html attributes 
        twitter_tokenizer = RegexpTokenizer(r'\w+') #Setup tokenizer          
        twitter_token = twitter_tokenizer.tokenize(twitter_text) #Tokenize words in string
        twitter_text = [word for word in twitter_token if word not in stop_words] #Remove stop words
        twitter_list.append(' '.join(twitter_text).replace('titl',''))

    return twitter_list

In [ ]:
def text_to_words (text):
    """This function converts sentences into a list of words"""
    words=[]

    for sentence in text:
        wordList = re.sub("[^\w]", " ",  sentence).split() #converts each text line into a list of words (creates a nested list)
        words.append(wordList)
    words= [item for sublist in words for item in sublist] #converts nested list into a 1D list to ease future implementation   
        
    return words

In [ ]:
def most_freq_words(words):
    counts = dict(Counter(words).most_common(70))

    labels, values = zip(*counts.items())

    # sort values in descending order
    indSort = np.argsort(values)[::-1]

    # rearrange data
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]

    indexes = np.arange(len(labels))

    bar_width = 0.1
    plt.figure(figsize=(20,10))
    plt.bar(indexes, values)
    plt.xticks(indexes + bar_width, labels,rotation='vertical')
    plt.title('Word counts for top 70 most repeated words')
    #sns.set(rc={'figure.figsize':(11.7,8.27)})
    plt.savefig(model_path + '/fig_most_freq_words.png', dpi=300)
    plt.show()
    


In [ ]:
def make_wordcloud(words):
    words_str = ' '.join(words) 
    wordcloud = WordCloud(background_color='white', width = 2000, height = 1000).generate(words_str)
    plt.figure(figsize=(20,10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title('Keywords')
    plt.axis("off")
    plt.savefig(model_path + '/fig_word_cloud.png', dpi=300)
    plt.show()
    

In [ ]:
def anon_col (df, cols):
    for col_name in cols:
        keys = {cats: i for i, cats in enumerate(df[col_name].unique())}
        df[col_name] = df[col_name].apply(lambda x: keys[x])
        
    return df

# 3. Defining global ML training parameters

In [ ]:
#Vectorization
bow = CountVectorizer(stop_words = "english", max_features= 300, ngram_range= (1,2)) #bag-of-words
tfidf = TfidfVectorizer(stop_words = "english", max_df=0.7, max_features= 300, ngram_range= (1,2)) #TFIDF

These are all the different classifiers and their hyper-parameters which will be used to train on. Modify as needed.

In [ ]:
#''
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20,30],
            'kernel': ['rbf','linear']
        }  
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='lbfgs',multi_class='multinomial'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes' : {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0,1]
        }
    },
    'xgboost' : {
        'model': xgb.XGBClassifier(),
        'params' : {
            'max_depth ':[10,20,50,100],
            'min_child_weight ':[5,10,15]
            
        }
        
    },
    'decision_tree' :{
        'model' : DecisionTreeClassifier(),
        'params' : {
            'max_depth' : [10,20,30,100],
            'criterion' : ['gini', 'entropy']
        }
    },
    'random_forrest' :{
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators' :[50, 100, 200],
            'max_depth' : [5, 10, 20, 100]
        }
    },
    'adaboost' : {
        'model' : AdaBoostClassifier(),
        'params' : {
            'n_estimators' :[200]
        }
    },
    'sgd' : {
        'model' : SGDClassifier(),
        'params' : {
            'loss' :['hinge'],
            'penalty': ['l2'],
            'max_iter' :[5,10,15]
        }
    },
    'k_nearest_neighbours' : {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [1,5,10]
        }
    }
}
#''

In [ ]:
'''
#This is a test param set. to be used when troubleshooting the notebook.

model_params = {
    'logistic_regression' : {
        'model': LogisticRegression(solver='lbfgs',multi_class='multinomial'),
        'params': {
            'C': [1]
        }
    }
}
'''

## 3. Creating the Main Dataframe

The annotated data will be imported. Each model will use this annotated data (dropping any non labelled Tweets).
For each model, a new DF will be created. e.g.: for Model 1: df_m1, for Model 2, df_m2 and so on.

In [ ]:
annotated_data = pd.read_excel(annotated_data_path) #This is the file with the annotated data
annotated_data.head(3)

In [ ]:
df_m1 = annotated_data
df_m1.target_label.astype(str)
df_m1 = df_m1[df_m1.target_label != 'TIE BREAK']
df_m1 = df_m1[df_m1.target_label != 5]
df_m1['target_label'] = df_m1['target_label'].fillna('')
df_m1 = df_m1[df_m1.target_label != '']

print(df_m1.target_label.value_counts().sort_index(axis=0))

print("Total samples: ", df_m1.target_label.value_counts().sum())
print("DF Shape: ",df_m1.shape)

X = df_m1.X
y = df_m1['target_label'].astype(int)

In [ ]:
#Word cloud on the cleaned text file
words = text_to_words(X)

In [ ]:
#Show most frequent words in dataset
most_freq_words(words)

In [ ]:
X = clean_X(X)
words = text_to_words(X)

In [ ]:
#Show most frequent words in dataset
most_freq_words(words)

In [ ]:
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
make_wordcloud(words)

# 4. Model Training

<b> Important note before you start: </b> 
<br>

Before model training is started, please note that this may take a long time (depending on how many trianing samples you have). The function <b> best_model_search() </b> which trains the models currently uses n_jobs = -1 which utilizes all the cores / threads of your computer's CPU. 
<br><br>
Should you wish to not allocate the whole CPU towards model training (which is recommended), please adjust the n_jobs to be equal to the number of threads that you can sufficiently allocate. for example, if you have a CPU with 4 threads, you may want to allocate only half of these to the training process, hence you would set n_jobs = 2. 
<br><br>
To check how many threads you have please check "Task Manager" on Windows OS devices, or "About this Mac" on Mac OS devices

### 4.1 Model 1
##### All classes with random undersampling of class 0

In [ ]:
#Undersampling the data based on the majority class
df_m1 = undersample_majority_classes(df_m1, m1_sampling)

In [ ]:
#define feature and target variables
X = df_m1.X
X = clean_X(X)
y = df_m1['target_label'].astype(int)

In [ ]:
model_dump_path = model_path + '\m1'

if not os.path.exists(model_dump_path):
    os.mkdir(model_dump_path)
    
start_time = time.time()

df_class, undersampling_est = best_model_search(X, y, bow, tfidf, model_params,model_dump_path)

elapsed_time = time.time() - start_time
elapsed_time

In [ ]:
for vec in ['bag-of-words','tfidf']:
    df_score = df_class[df_class['vect'] == vec]
    plot_histogram_data(df_score)
    df_score.to_excel(model_dump_path + '/scores_' + vec + 'df_score_m1.xlsx',encoding = 'utf-8-sig')
    plt.savefig(model_dump_path + '/fig_' + vec + '_results_m1.png', dpi=300)

In [ ]:
df_score

In [ ]:
X_ft = bow.fit_transform(X).toarray()
y_pred = cross_val_predict(undersampling_est[1], X_ft, y, cv=10)
cm = confusion_matrix(y, y_pred)
ax = plt.axes()
df_cm = sb.heatmap(pd.DataFrame(cm),cmap='Blues',annot=True,ax=ax)
ax.set_title('Best Model - Model 1')
plt.savefig(model_dump_path + '/fig_' + vec + '_results_m1_CM.png', dpi=300)

### 4.2 Model 2
##### Removing class 4, random undersampling of class 0. Total of four classes

In [ ]:
df_m2 = annotated_data
df_m2.target_label.astype(str)
df_m2 = df_m2[df_m2.target_label != 'TIE BREAK']
df_m2 = df_m2[df_m2.target_label != 5]
df_m2['target_label'] = df_m2['target_label'].fillna('')
df_m2 = df_m2[df_m2.target_label != '']

print(df_m2.target_label.value_counts().sort_index(axis=0))

print("Total samples: ", df_m2.target_label.value_counts().sum())
print("DF Shape: ",df_m2.shape)

In [ ]:
# Dropping class 4
df_m2 = df_m2[df_m2.target_label != 4]
df_m2.target_label.value_counts().sort_index(axis=0)

In [ ]:
#undersample class 0
df_m2 = undersample_majority_classes(df_m2, m2_sampling)

In [ ]:
X = df_m2.X
X = clean_X(X)
y = df_m2['target_label'].astype(int)

In [ ]:
model_dump_path = model_path + '\m2'
if not os.path.exists(model_dump_path):
    os.mkdir(model_dump_path)
    
start_time = time.time()

df_class, undersampling_est = best_model_search(X, y, bow, tfidf, model_params,model_dump_path)

elapsed_time = time.time() - start_time
elapsed_time


In [ ]:
print(elapsed_time)

In [ ]:
for vec in ['bag-of-words','tfidf']:
    df_score = df_class[df_class['vect'] == vec]
    plot_histogram_data(df_score)
    df_score.to_excel(model_dump_path + '/scores_' + vec + 'df_score_m2.xlsx',encoding = 'utf-8-sig')
    plt.savefig(model_dump_path + '/fig_' + vec + '_results_m2.png', dpi=300)

In [ ]:
df_score

In [ ]:
X_ft = bow.fit_transform(X).toarray()
y_pred = cross_val_predict(undersampling_est[1], X_ft, y, cv=10)
cm = confusion_matrix(y, y_pred)
ax = plt.axes()
df_cm = sb.heatmap(pd.DataFrame(cm),cmap='Blues',annot=True,ax=ax)
ax.set_title('Best Model - Model 2')
plt.savefig(model_dump_path + '/fig_' + vec + '_results_m2_CM.png', dpi=300)

### 4.3 Model 3
##### Converting class 4 into class 1, random undersampling of class 0 (four classes in total)

In [ ]:
df_m3 = annotated_data
df_m3.target_label.astype(str)
df_m3 = df_m3[df_m3.target_label != 'TIE BREAK']
df_m3 = df_m3[df_m3.target_label != 5]
df_m3['target_label'] = df_m3['target_label'].fillna('')
df_m3 = df_m3[df_m3.target_label != '']

print(df_m3.target_label.value_counts().sort_index(axis=0))

print("Total samples: ", df_m3.target_label.value_counts().sum())
print("DF Shape: ",df_m3.shape)


In [ ]:
# Merging class 4 to class 1
df_m3['target_label'].replace({4: 1}, inplace=True)
df_m3.target_label.value_counts().sort_index(axis=0)

In [ ]:
#undersample class 0
df_m3 = undersample_majority_classes(df_m3, m3_sampling)

In [ ]:
X = df_m3.X
X = clean_X(X)
y = df_m3['target_label'].astype(int)

In [ ]:
model_dump_path = model_path + '\m3'
if not os.path.exists(model_dump_path):
    os.mkdir(model_dump_path)
start_time = time.time()

df_class, undersampling_est = best_model_search(X, y, bow, tfidf, model_params,model_dump_path)

elapsed_time = time.time() - start_time
elapsed_time


In [ ]:
print(elapsed_time)

In [ ]:
for vec in ['bag-of-words','tfidf']:
    df_score = df_class[df_class['vect'] == vec]
    plot_histogram_data(df_score)
    df_score.to_excel(model_dump_path + '/scores_' + vec + 'df_score_m3.xlsx',encoding = 'utf-8-sig')
    plt.savefig(model_dump_path + '/fig_' + vec + '_results_m3.png', dpi=300)

In [ ]:
df_score

In [ ]:
X_ft = bow.fit_transform(X).toarray()
y_pred = cross_val_predict(undersampling_est[1], X_ft, y, cv=10)
cm = confusion_matrix(y, y_pred)
ax = plt.axes()
df_cm = sb.heatmap(pd.DataFrame(cm),cmap='Blues',annot=True,ax=ax)
ax.set_title('Best Model - Model 3')
plt.savefig(model_dump_path + '/fig_' + vec + '_results_m3_CM.png', dpi=300)

### 4.4 Model 4
##### Removing class 4, converting class 3 into class 2 (three classes in total)

In [ ]:
df_m4 = annotated_data
df_m4.target_label.astype(str)
df_m4 = df_m4[df_m4.target_label != 'TIE BREAK']
df_m4 = df_m4[df_m4.target_label != 5]
df_m4['target_label'] = df_m4['target_label'].fillna('')
df_m4 = df_m4[df_m4.target_label != '']

print(df_m4.target_label.value_counts().sort_index(axis=0))

print("Total samples: ", df_m4.target_label.value_counts().sum())
print("DF Shape: ",df_m4.shape)


In [ ]:
#Drop class 4
df_m4 = df_m4[df_m4.target_label != 4]
df_m4.target_label.value_counts().sort_index(axis=0)

In [ ]:
#Merge class 3 into class 2
df_m4.target_label.replace(to_replace =3, value = 2,inplace=True)
df_m4.target_label.value_counts().sort_index(axis=0)

In [ ]:
#undersample class 0
df_m4 = undersample_majority_classes(df_m4, m4_sampling)

In [ ]:
X = df_m4.X
X = clean_X(X)
y = df_m4['target_label'].astype(int)

In [ ]:
model_dump_path = model_path + '\m4'
if not os.path.exists(model_dump_path):
    os.mkdir(model_dump_path)
    
start_time = time.time()

df_class, undersampling_est = best_model_search(X, y, bow, tfidf, model_params,model_dump_path)

elapsed_time = time.time() - start_time
elapsed_time


In [ ]:
print(elapsed_time)

In [ ]:
for vec in ['bag-of-words','tfidf']:
    df_score = df_class[df_class['vect'] == vec]
    plot_histogram_data(df_score)
    df_score.to_excel(model_dump_path + '/scores_' + vec + 'df_score_m4.xlsx',encoding = 'utf-8-sig')
    plt.savefig(model_dump_path + '/fig_' + vec + '_results_m4.png', dpi=300)

In [ ]:
df_score

In [ ]:
X_ft = bow.fit_transform(X).toarray()
y_pred = cross_val_predict(undersampling_est[1], X_ft, y, cv=10)
cm = confusion_matrix(y, y_pred)
ax = plt.axes()
df_cm = sb.heatmap(pd.DataFrame(cm),cmap='Blues',annot=True,ax=ax)
ax.set_title('Best Model - Model 4')
plt.savefig(model_dump_path + '/fig_' + vec + '_results_m4_CM.png', dpi=300)

### 4.5 Model 5
##### Converting class 4 to class 1, converting class 3 to 2 (three classes in total)

In [ ]:
df_m5 = annotated_data
df_m5.target_label.astype(str)
df_m5 = df_m5[df_m5.target_label != 'TIE BREAK']
df_m5 = df_m5[df_m5.target_label != 5]
df_m5['target_label'] = df_m5['target_label'].fillna('')
df_m5 = df_m5[df_m5.target_label != '']

print(df_m5.target_label.value_counts().sort_index(axis=0))

print("Total samples: ", df_m5.target_label.value_counts().sum())
print("DF Shape: ",df_m5.shape)


In [ ]:
# Merging class 4 to class 1
df_m5.target_label.replace(to_replace =4, value = 1,inplace=True)
df_m5.target_label.replace(to_replace =3, value = 2,inplace=True)

In [ ]:
df_m5.target_label.value_counts().sort_index(axis=0)

In [ ]:
#undersample class 0
df_m5 = undersample_majority_classes(df_m5, m5_sampling)

In [ ]:
X = df_m5.X
X = clean_X(X)
y = df_m5['target_label'].astype(int)

In [ ]:
model_dump_path = model_path + '\m5'
if not os.path.exists(model_dump_path):
    os.mkdir(model_dump_path)

start_time = time.time()

df_class, undersampling_est = best_model_search(X, y, bow, tfidf, model_params,model_dump_path)

elapsed_time = time.time() - start_time
elapsed_time


In [ ]:
print(elapsed_time)

In [ ]:
for vec in ['bag-of-words','tfidf']:
    df_score = df_class[df_class['vect'] == vec]
    plot_histogram_data(df_score)
    df_score.to_excel(model_dump_path + '/scores_' + vec + 'df_score_m5.xlsx',encoding = 'utf-8-sig')
    plt.savefig(model_dump_path + '/fig_' + vec + '_results_m5.png', dpi=300)

In [ ]:
df_score

In [ ]:
X_ft = bow.fit_transform(X).toarray()
y_pred = cross_val_predict(undersampling_est[1], X_ft, y, cv=10)
cm = confusion_matrix(y, y_pred)
ax = plt.axes()
df_cm = sb.heatmap(pd.DataFrame(cm),cmap='Blues',annot=True,ax=ax)
ax.set_title('Best Model - Model 5')
plt.savefig(model_dump_path + '/fig_' + vec + '_results_m5_CM.png', dpi=300)

In [ ]:
global_elapsed_time = time.time() - global_start_time
print(global_elapsed_time)

---

# 5. Model Predictions
 <b> USER INPUT IS REQUIRED</b> 
This section is for applying the best model's classifier to predict label for the entire dataset.
For the purpose of this project, we decided to limit it to 4 classes - specifically <b>model 3's </b> classifier with the best F1 Score was used which is <b> Logistic Regression </b> </b> using <b> TFIDF </b> with the following params: <b> {'C': 1}.</b>
We leave it to the user to identify the best model and change the file path as needed.

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(best_model, 'rb'))

In [ ]:
dashboard_data = annotated_data

In [ ]:
X = dashboard_data.X
X = clean_X(X)

In [ ]:
X_ft = tfidf.fit_transform(X).toarray()

In [ ]:
y_pred = loaded_model.predict(X_ft)
len(y_pred)

In [ ]:
dashboard_data['y_pred'] = y_pred

In [ ]:
dashboard_data.drop(columns=['X'], inplace = True)

In [ ]:
dashboard_data

In [ ]:
dashboard_data.to_excel(dashboard_file_path + "/dashboard_data_predictions.xlsx",encoding = 'utf-8-sig')